In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade scikit-learn

In [2]:
import os
import cv2
import numpy as np
from skimage import feature
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import joblib
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
def load_and_preprocess_images(folder_path):
    images = []
    labels = []

    for file in os.listdir(folder_path):
        if file.endswith((".jpg", ".png")):
            image_path = os.path.join(folder_path, file)
            image = cv2.imread(image_path, 0)
            image = cv2.resize(image, (224, 224))
            images.append(image)
            labels.append(os.path.basename(folder_path))

    return images, labels


In [4]:
def extract_lbp_features(image):
    lbp = feature.local_binary_pattern(image, P=8, R=1, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 60), range=(0, 59))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-5)
    #print(hist)
    return hist

In [5]:
from skimage.feature import hog
def extract_hog_features(image):
    _, hog_features = hog(image, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(1, 1), visualize=True)
    return hog_features


In [6]:
def extract_combined_features(image):
    lbp_features = extract_lbp_features(image)
    hog_features = extract_hog_features(image)

    lbp_features = lbp_features.flatten()
    hog_features = hog_features.flatten()

    combined_features = np.concatenate([lbp_features, hog_features])
    return combined_features

In [7]:
real_images, real_labels = load_and_preprocess_images("/content/drive/MyDrive/face/real_and_morphed_face/training_real")
morph_images, morph_labels = load_and_preprocess_images("/content/drive/MyDrive/face/real_and_morphed_face/training_morphed")

In [8]:
X = []
y = []

for image, label in zip(real_images, real_labels):
    features = extract_combined_features(image)
    X.append(features)
    y.append(label)

for image, label in zip(morph_images, morph_labels):
    features = extract_combined_features(image)
    X.append(features)
    y.append(label)

X = np.array(X)
y = np.array(y)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from sklearn.model_selection import GridSearchCV

In [11]:
param_grid = {'C': [0.1, 1], 'kernel': ['linear', 'rbf']}


In [12]:
grid_search = GridSearchCV(SVC(), param_grid, cv=5, n_jobs=-1)


In [13]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {'C': [1], 'kernel': ['linear', 'rbf']}

# RandomizedSearchCV for faster exploration of hyperparameter space
random_search = RandomizedSearchCV(SVC(), param_distributions=param_grid, n_iter=10, cv=5, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

# Print the best parameters
print("Best Parameters:", random_search.best_params_)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Parameters: {'kernel': 'rbf', 'C': 1}


In [15]:
best_classifier = random_search.best_estimator_

In [16]:
best_classifier.fit(X_train, y_train)

SVC(C=1)

In [17]:
y_pred = best_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy *100 :.2f}%")

Accuracy: 67.24%


In [18]:
scaler_filename = '/content/drive/MyDrive/face/scaler.joblib'
joblib.dump(scaler, scaler_filename)
print(f"Scaler saved to {scaler_filename}")

Scaler saved to /content/drive/MyDrive/face/scaler.joblib


In [19]:
model_filename = "/content/drive/MyDrive/face/detector.pkl"
joblib.dump(best_classifier, model_filename)

['/content/drive/MyDrive/face/detector.pkl']

In [20]:
loaded_classifier = joblib.load(model_filename)

In [21]:
def predict_real_or_morph(input_image):
    input_features = extract_combined_features(input_image)
    input_features = scaler.transform([input_features])
    prediction = loaded_classifier.predict(input_features)
    return "Real" if prediction[0] == "real" else "Morphed"

In [22]:
input_image_path = "/content/drive/MyDrive/face/real_and_morphed_face/training_morphed/easy_173_0111.jpg"
input_image = cv2.imread(input_image_path, 0)
input_image = cv2.resize(input_image, (224, 224))
result = predict_real_or_morph(input_image)
print(f"The input image is predicted as: {result}")

The input image is predicted as: Morphed
